In [ ]:
#Dataset Link: https://www.kaggle.com/datasets/abhikjha/imdb-wiki-faces-dataset
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime

# Define the dataset directory path
data_dir =  r'E:\archive\imdb_crop\00'  

# Preprocess function to load and resize the image
def preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    img = img.astype('float32') / 255.0  # Normalize to [0, 1]
    return img

# Extract age from filename
def extract_age_from_filename(filename):
    # Assuming the filename follows the format 'nm0000099_rm11132416_1968-4-8_2010.jpg'
    parts = filename.split('_')
    dob_str = parts[2]  # Date of birth part (e.g., '1968-4-8')
    photo_year_str = parts[3].split('.')[0]  # Photo taken year part (e.g., '2010')

    # Parse the date of birth and photo year
    dob = datetime.strptime(dob_str, "%Y-%m-%d")
    photo_year = int(photo_year_str)

    # Calculate age
    age = photo_year - dob.year
    return age

# Load images and ages from the directory
def load_images_and_labels(data_dir):
    images = []
    ages = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Check image extensions
            image_path = os.path.join(data_dir, filename)
            try:
                img = preprocess_image(image_path)
                age = extract_age_from_filename(filename)
                images.append(img)
                ages.append(age)
            except Exception as e:
                print(f"Error processing file {filename}: {e}")
                continue
    return np.array(images), np.array(ages)

# Load images and ages
images, ages = load_images_and_labels(data_dir)
print(ages)
print(images)
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, ages, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the pre-trained VGG16 model (part of VGGFace)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='linear')(x)  # Single neuron for regression (age prediction)

# Create the final model
model = Model(inputs=base_model.input, outputs=x)

# Compile the model using the correct argument
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error', metrics=['mae'])

# Show the model summary
model.summary()


In [ ]:
# Define batch size and epochs
batch_size = 32
epochs = 10

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)


In [ ]:
# Evaluate the model
loss, mae = model.evaluate(X_val, y_val, verbose=1)
print(f"Validation Mean Absolute Error: {mae:.2f} years")


In [ ]:
# Unfreeze the last few layers of the base model
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Re-compile the model with a smaller learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='mean_squared_error', metrics=['mae'])

# Fine-tune the model
fine_tune_epochs = 5
history_fine = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=fine_tune_epochs,
    batch_size=batch_size,
    verbose=1
)


In [ ]:
model.save('vggface_age_detection_model.h5')


In [ ]:
# GUI of Age Detection

import tkinter as tk
from tkinter import filedialog
from tkinter import Label
from PIL import Image, ImageTk
import numpy as np
import cv2
import tensorflow as tf



In [ ]:
# Load the trained model
model = tf.keras.models.load_model('vggface_age_detection_model.h5')


In [ ]:
# Create the main window
window = tk.Tk()
window.title("Age Detection from Image")
window.geometry("600x400")

# Function to select an image file
def load_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # Load and display the image
        img = Image.open(file_path)
        img = img.resize((224, 224))  # Resize for display and model input
        img_tk = ImageTk.PhotoImage(img)
        image_label.config(image=img_tk)
        image_label.image = img_tk

        # Preprocess image for model prediction
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        img_array = img_array.astype('float32') / 255.0  # Normalize

        # Predict age
        predicted_age = model.predict(img_array)
        age_label.config(text=f"Predicted Age: {int(predicted_age[0])} years")

# Add widgets to the window
image_label = Label(window)
image_label.pack(pady=20)

age_label = Label(window, text="Predicted Age: N/A", font=('Helvetica', 14))
age_label.pack(pady=10)

load_button = tk.Button(window, text="Load Image", command=load_image)
load_button.pack(pady=20)


In [ ]:
# Start the GUI loop
window.mainloop()
